In [1]:
import pandas as pd
import mod_sql

In [2]:
## 클래스 선언
_db = mod_sql.Database()

### emp table
>   사원 정보를 담고 있는 테이블

### dept table
>   부서 정보를 담고 있는 테이블

In [3]:
sql = """
select * from emp
"""
_db.executeAll(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902,1980-12-17,800.0,,20
1,7499,ALLEN,SALESMAN,7698,1981-02-20,1600.0,300.00,30
2,7521,WARD,SALESMAN,7698,1981-02-22,1250.0,500.00,30
3,7566,JONES,MANAGER,7839,1981-04-02,2975.0,,20
4,7654,MARTIN,SALESMAN,7698,1981-09-28,1250.0,1400.00,30
5,7698,BLAKE,MANAGER,7839,1981-05-01,2850.0,,30
6,7782,CLARK,MANAGER,7839,1981-06-09,2450.0,,10
7,7788,SCOTT,ANALYST,7566,1987-06-28,3000.0,,20
8,7839,KING,PRESIDENT,0,1981-11-17,5000.0,,10
9,7844,TURNER,SALESMAN,7698,1981-09-08,1500.0,0.00,30


### 부서의 이름이 'sales'인 소속 사원들을 출력
1. dept 테이블에서 부서이름이 sales인 DEPTNO을 가지고 온다. 
2. emp 테이블에서 DEPTNO 1번 과정에서 나온 결과값을 기준으로 사원 정보 출력

In [4]:
sql = """
select * from emp 
WHERE DEPTNO = (
    select DEPTNO from dept 
    where DNAME = 'SALES'
)
"""
_db.executeAll(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7499,ALLEN,SALESMAN,7698,1981-02-20,1600.0,300.00,30
1,7521,WARD,SALESMAN,7698,1981-02-22,1250.0,500.00,30
2,7654,MARTIN,SALESMAN,7698,1981-09-28,1250.0,1400.00,30
3,7698,BLAKE,MANAGER,7839,1981-05-01,2850.0,,30
4,7844,TURNER,SALESMAN,7698,1981-09-08,1500.0,0.00,30
5,7900,JAMES,CLERK,7698,1981-12-03,950.0,,30


#### 부서번호가 10, 30 인 사원 정보를 출력

In [5]:
sql = """
select * from emp 
where DEPTNO = 10 or DEPTNO = 30
"""

sql_2 = """
select * from emp 
where DEPTNO in (10, 30)
"""
## sql과 sql_2의 쿼리문은 같은 값을 출력한다. 
_db.executeAll(sql_2)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7499,ALLEN,SALESMAN,7698,1981-02-20,1600.0,300.00,30
1,7521,WARD,SALESMAN,7698,1981-02-22,1250.0,500.00,30
2,7654,MARTIN,SALESMAN,7698,1981-09-28,1250.0,1400.00,30
3,7698,BLAKE,MANAGER,7839,1981-05-01,2850.0,,30
4,7782,CLARK,MANAGER,7839,1981-06-09,2450.0,,10
5,7839,KING,PRESIDENT,0,1981-11-17,5000.0,,10
6,7844,TURNER,SALESMAN,7698,1981-09-08,1500.0,0.00,30
7,7900,JAMES,CLERK,7698,1981-12-03,950.0,,30
8,7934,MILLER,CLERK,7782,1982-01-23,1300.0,,10


### job_code 새로운 파생변수 생성
1. salesman -> 1 , manager -> 2, 그외의 직업은 3
2. EMPNO, ENAME, JOB, job_code 출력

In [6]:
sql = """
SELECT EMPNO, ENAME, JOB, 
    CASE 
        WHEN JOB = 'SALESMAN'
        THEN '1' 
        WHEN JOB = 'MANAGER'
        THEN '2' 
        ELSE '3' 
        END AS JOB_CODE 
FROM emp
"""

_db.executeAll(sql)

,EMPNO,ENAME,JOB,JOB_CODE
0,7369,SMITH,CLERK,3
1,7499,ALLEN,SALESMAN,1
2,7521,WARD,SALESMAN,1
3,7566,JONES,MANAGER,2
4,7654,MARTIN,SALESMAN,1
5,7698,BLAKE,MANAGER,2
6,7782,CLARK,MANAGER,2
7,7788,SCOTT,ANALYST,3
8,7839,KING,PRESIDENT,3
9,7844,TURNER,SALESMAN,1


### 위에서 SQL구문으로 파생변수를 생성하였는데 pandas를 이용하여 같은 값을 출력해보시오

In [7]:
sql = """
    select * from emp
"""
df = _db.executeAll(sql)

## 파생변수 JOB_CODE 생성 (case1)
def change(x):
    if x == "SALESMAN":
        return "1"
    elif x == "MANAGER":
        return "2"
    else:
        return "3"
df["JOB_CODE"] = df["JOB"].apply(change)
df.loc[:,["EMPNO", "ENAME", "JOB", "JOB_CODE"]]

## case2
df["JOB_CODE"] = df["JOB"].apply(lambda x : "1" if (x == "SALESMAN") 
else ("2" if (x == "MANAGER") else "3"))
df.loc[:,["EMPNO", "ENAME", "JOB", "JOB_CODE"]]


,EMPNO,ENAME,JOB,JOB_CODE
0,7369,SMITH,CLERK,3
1,7499,ALLEN,SALESMAN,1
2,7521,WARD,SALESMAN,1
3,7566,JONES,MANAGER,2
4,7654,MARTIN,SALESMAN,1
5,7698,BLAKE,MANAGER,2
6,7782,CLARK,MANAGER,2
7,7788,SCOTT,ANALYST,3
8,7839,KING,PRESIDENT,3
9,7844,TURNER,SALESMAN,1


In [8]:
## concat -> 결합 sql에서도 concat
sql = """
select concat(ENAME, '(', JOB , ')') as ENAME from emp
"""
_db.executeAll(sql)

,ENAME
0,SMITH(CLERK)
1,ALLEN(SALESMAN)
2,WARD(SALESMAN)
3,JONES(MANAGER)
4,MARTIN(SALESMAN)
5,BLAKE(MANAGER)
6,CLARK(MANAGER)
7,SCOTT(ANALYST)
8,KING(PRESIDENT)
9,TURNER(SALESMAN)


In [9]:
df["ENAME_"] = df["ENAME"] + '(' + df["JOB"] + ')'
df["ENAME_"]

0         SMITH(CLERK)
1      ALLEN(SALESMAN)
2       WARD(SALESMAN)
3       JONES(MANAGER)
4     MARTIN(SALESMAN)
5       BLAKE(MANAGER)
6       CLARK(MANAGER)
7       SCOTT(ANALYST)
8      KING(PRESIDENT)
9     TURNER(SALESMAN)
10        ADAMS(CLERK)
11        JAMES(CLERK)
12       FORD(ANALYST)
13       MILLER(CLERK)
Name: ENAME_, dtype: object

In [10]:
## if문 사용한 sql 쿼리문
sql = """
select SAL, if(SAL >= 2000, "HIGH", "LOW") as `H/L` from emp
"""
_db.executeAll(sql)

,SAL,H/L
0,800.0,LOW
1,1600.0,LOW
2,1250.0,LOW
3,2975.0,HIGH
4,1250.0,LOW
5,2850.0,HIGH
6,2450.0,HIGH
7,3000.0,HIGH
8,5000.0,HIGH
9,1500.0,LOW


In [11]:
### pandas를 이용해서 위 sql문과 같은 작업을 실행.
### case1
df["H/L"] = df["SAL"].apply(lambda x : "HIGH" if x >= 2000 else "LOW")
df[["SAL", "H/L"]]

### case2
def sal_change(x):
    if x >= 2000:
        return "HIGH"
    else:
        return "LOW"
df["H/L"] = df["SAL"].apply(sal_change)

In [ ]:
### REPLACE -> 값을 대체. replace("변경이 될 단어", "변경 할 단어")
sql = """
select JOB, replace(JOB, "SALESMAN", "SALES") FROM emp
"""
_db.executeAll(sql)

In [16]:
### pandas replace 실행. 파생변수 생성하여 작업(job_)
df["JOB_"] = df["JOB"].str.replace("SALESMAN", "SALES")
df[["JOB", "JOB_"]]

,JOB,JOB_
0,CLERK,CLERK
1,SALESMAN,SALES
2,SALESMAN,SALES
3,MANAGER,MANAGER
4,SALESMAN,SALES
5,MANAGER,MANAGER
6,MANAGER,MANAGER
7,ANALYST,ANALYST
8,PRESIDENT,PRESIDENT
9,SALESMAN,SALES


In [23]:
### join -> left join
sql = """
select * from emp left join dept on emp.DEPTNO = dept.DEPTNO
"""
_db.executeAll(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,dept.DEPTNO,DNAME,LOC
0,7369,SMITH,CLERK,7902,1980-12-17,800.0,,20,20.0,RESEARCH,DALLAS
1,7499,ALLEN,SALESMAN,7698,1981-02-20,1600.0,300.00,30,30.0,SALES,CHICAGO
2,7521,WARD,SALESMAN,7698,1981-02-22,1250.0,500.00,30,30.0,SALES,CHICAGO
3,7566,JONES,MANAGER,7839,1981-04-02,2975.0,,20,20.0,RESEARCH,DALLAS
4,7654,MARTIN,SALESMAN,7698,1981-09-28,1250.0,1400.00,30,30.0,SALES,CHICAGO
5,7698,BLAKE,MANAGER,7839,1981-05-01,2850.0,,30,30.0,SALES,CHICAGO
6,7782,CLARK,MANAGER,7839,1981-06-09,2450.0,,10,10.0,ACCOUNTING,NEW YORK
7,7788,SCOTT,ANALYST,7566,1987-06-28,3000.0,,20,20.0,RESEARCH,DALLAS
8,7839,KING,PRESIDENT,0,1981-11-17,5000.0,,10,10.0,ACCOUNTING,NEW YORK
9,7844,TURNER,SALESMAN,7698,1981-09-08,1500.0,0.00,30,30.0,SALES,CHICAGO


In [24]:
sql = """
select * from emp right join dept on emp.DEPTNO = dept.DEPTNO
"""
_db.executeAll(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,dept.DEPTNO,DNAME,LOC
0,7934.0,MILLER,CLERK,7782.0,1982-01-23,1300.0,,10.0,10,ACCOUNTING,NEW YORK
1,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,,10.0,10,ACCOUNTING,NEW YORK
2,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,,10.0,10,ACCOUNTING,NEW YORK
3,7902.0,FORD,ANALYST,7566.0,1981-12-03,3000.0,,20.0,20,RESEARCH,DALLAS
4,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,,20.0,20,RESEARCH,DALLAS
5,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,,20.0,20,RESEARCH,DALLAS
6,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,,20.0,20,RESEARCH,DALLAS
7,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,,20.0,20,RESEARCH,DALLAS
8,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,,30.0,30,SALES,CHICAGO
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.00,30.0,30,SALES,CHICAGO


In [22]:
sql = """
insert into emp values (%s, %s, %s, %s, %s, %s, %s, %s)
"""
values = [8000, "test", "CEO", 0, "1988-02-01", 3500.0, 100, 50]
_db.execute(sql, values)
_db.commit()

In [25]:
### inner join 으로 데이터를 결합 
### 결과 값이 어떻게 나오는지 확인!
sql = """
select * from emp inner join dept on emp.DEPTNO = dept.DEPTNO
"""
_db.executeAll(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,dept.DEPTNO,DNAME,LOC
0,7369,SMITH,CLERK,7902,1980-12-17,800.0,,20,20,RESEARCH,DALLAS
1,7499,ALLEN,SALESMAN,7698,1981-02-20,1600.0,300.00,30,30,SALES,CHICAGO
2,7521,WARD,SALESMAN,7698,1981-02-22,1250.0,500.00,30,30,SALES,CHICAGO
3,7566,JONES,MANAGER,7839,1981-04-02,2975.0,,20,20,RESEARCH,DALLAS
4,7654,MARTIN,SALESMAN,7698,1981-09-28,1250.0,1400.00,30,30,SALES,CHICAGO
5,7698,BLAKE,MANAGER,7839,1981-05-01,2850.0,,30,30,SALES,CHICAGO
6,7782,CLARK,MANAGER,7839,1981-06-09,2450.0,,10,10,ACCOUNTING,NEW YORK
7,7788,SCOTT,ANALYST,7566,1987-06-28,3000.0,,20,20,RESEARCH,DALLAS
8,7839,KING,PRESIDENT,0,1981-11-17,5000.0,,10,10,ACCOUNTING,NEW YORK
9,7844,TURNER,SALESMAN,7698,1981-09-08,1500.0,0.00,30,30,SALES,CHICAGO


In [26]:
### pandas merge()함수를 이용하여 left , right, inner 3가지의 경우를 모두 출력
### sql에서 했던 조건과 같다. (부서이름을 기준으로 merge 사용)
sql = """
select * from emp
"""
df = _db.executeAll(sql)

In [27]:
sql = """
select * from dept
"""
df_2 = _db.executeAll(sql)

In [ ]:
## left join
pd.merge(df, df_2, on="DEPTNO", how='left')

In [ ]:
## right join
pd.merge(df, df_2, on="DEPTNO", how='right')

In [ ]:
## inner join 
pd.merge(df, df_2, on = 'DEPTNO', how='inner')

In [34]:
## full outer join
pd.merge(df, df_2, on="DEPTNO", how="outer")

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,DNAME,LOC
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,,20,RESEARCH,DALLAS
1,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,,20,RESEARCH,DALLAS
2,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,,20,RESEARCH,DALLAS
3,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,,20,RESEARCH,DALLAS
4,7902.0,FORD,ANALYST,7566.0,1981-12-03,3000.0,,20,RESEARCH,DALLAS
5,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.00,30,SALES,CHICAGO
6,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.00,30,SALES,CHICAGO
7,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.00,30,SALES,CHICAGO
8,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,,30,SALES,CHICAGO
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.00,30,SALES,CHICAGO


In [ ]:
#case1
sql = """
select * from emp 
where SAL >= 2000 and SAL <= 5000
"""
_db.executeAll(sql)

In [37]:
#case2
sql ="""
select * from emp 
where SAL between 2000 and 5000
"""
_db.executeAll(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7566,JONES,MANAGER,7839,1981-04-02,2975.0,,20
1,7698,BLAKE,MANAGER,7839,1981-05-01,2850.0,,30
2,7782,CLARK,MANAGER,7839,1981-06-09,2450.0,,10
3,7788,SCOTT,ANALYST,7566,1987-06-28,3000.0,,20
4,7839,KING,PRESIDENT,0,1981-11-17,5000.0,,10
5,7902,FORD,ANALYST,7566,1981-12-03,3000.0,,20
6,8000,test,CEO,0,1988-02-01,3500.0,100,50
